# Supabase 

[**Supabase**](https://supabase.com/) is an open source backend-as-a-service platform. We will use the [free Postgres database](https://supabase.com/docs/guides/database/overview) that is offered with each project to store our SQL tables. 

You will need to create an account, and will also be prompted to create an "organization". Within this organization, you can create projects. Start a new project, give it name and a password for the database. This password allows access to your database, it **must be kept private**. Save it somewhere for now.

From your dashboard, you can have a look at your database by navigating to "database" from the menu on the left. At the moment, your database is empty. Supabase provide a UI and and API to manage the database, but we are going to connect it directly to our Python codebase. To do this, click "Connect" on the top menu bar and copy the "Direct connection" string. 

Create a `.env` file and set the connection string to a variable called `DB_CONN`, then make sure your env file is in your `.gitignore`! Now replace the part of the string `[YOUR-PASSWORD]` with the password you saved earlier.

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

dbconn = os.getenv("DBCONN")

Install [`psycopg`](https://www.psycopg.org/docs/) through Anaconda - you must have the [`conda-forge` channel](https://anaconda.org/conda-forge/psycopg/) activated for the most recent version (recommended). This package lets us connect and make queries to a PostgreSQL database. 

We're then going to follow the steps in the [psycopg documentation](https://www.psycopg.org/docs/usage.html) to establish the database connection, create a [cursor](https://www.psycopg.org/psycopg3/docs/api/cursors.html), and then execute some queries. You will need the connection to `.commit()` some changes before you see them applied to your database. 

Note that since your code must now send your queries to a Cloud service, there might be some delay in the response. Also be aware that we are no longer using SQLite, so there might also be some small differences in the SQL queries. Most notably, the [datatype system](https://neon.tech/postgresql/postgresql-tutorial/postgresql-data-types) for PostgreSQL is much more extensive!

In [11]:
import psycopg

# establish database connection using connection string from .env
conn = psycopg.connect(dbconn)

# create a cursor
cur = conn.cursor()

In [ ]:
cur.execute('''
    CREATE TABLE IF NOT EXISTS weather_data(
        date TIMESTAMP,
        city VARCHAR,
        temp FLOAT,
        feels FLOAT,
        description VARCHAR,
        PRIMARY KEY(date, city)
    );
''')

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=db.bxzxrmyehjbtwvopzpyh.supabase.co database=postgres) at 0x1bd5e035c10>

In [13]:
# commit to all executed queries
conn.commit()

In [14]:
# close the cursor and sever connection
cur.close()
conn.close()

The table I've created is to hold data fetched from the ["current weather" endpoint](https://openweathermap.org/current) from the Open Weather map API. We can customize the endpoint to search for the current weather in various [cities](https://openweathermap.org/current#name). 

In [8]:
api_key = os.getenv("WEATHER_APIKEY")

In [9]:
import requests
import json # this is just to make the print statement more readable

city = "berlin"
endpoint = f"http://api.openweathermap.org/data/2.5/weather?units=metric&appid={api_key}&q={city}"

response = requests.get(endpoint)
response_json = response.json()
print(json.dumps(response_json, indent=4))

{
    "coord": {
        "lon": 13.4105,
        "lat": 52.5244
    },
    "weather": [
        {
            "id": 801,
            "main": "Clouds",
            "description": "few clouds",
            "icon": "02d"
        }
    ],
    "base": "stations",
    "main": {
        "temp": 21.31,
        "feels_like": 21.17,
        "temp_min": 19.9,
        "temp_max": 23.37,
        "pressure": 1014,
        "humidity": 64,
        "sea_level": 1014,
        "grnd_level": 1008
    },
    "visibility": 10000,
    "wind": {
        "speed": 3.09,
        "deg": 190
    },
    "clouds": {
        "all": 20
    },
    "dt": 1748866807,
    "sys": {
        "type": 2,
        "id": 2091794,
        "country": "DE",
        "sunrise": 1748832528,
        "sunset": 1748892015
    },
    "timezone": 7200,
    "id": 2950159,
    "name": "Berlin",
    "cod": 200
}


Assuming my request was successful and I have a proper response, I will separate just the values I want to save into the database. JSON follows the same datatype pattern as Python dictionaries, so we can index the keys or use the [`.get()`](https://www.w3schools.com/python/ref_dictionary_get.asp) method to target the desired values. 

In [10]:
import datetime as dt

if response.status_code == 200:
  weather_date = dt.datetime.fromtimestamp(response_json["dt"])
  weather_city = response_json["name"]
  weather_temp = response_json["main"]["temp"]
  weather_feels = response_json["main"]["feels_like"]
  weather_description = response_json["weather"][0]["description"]

  print(weather_date, weather_city, weather_temp, weather_feels, weather_description)


2025-06-02 14:20:07 Berlin 21.31 21.17 few clouds


We then want to execute a query to `INSERT` rows into our SQL database. We are going to use [**parameter placeholders**](https://www.psycopg.org/docs/usage.html#passing-parameters-to-sql-queries) to do this. Parameter placeholders are exactly what they sound like - placeholder values that represent values to be sent along with the query. SQL libraries utilize this to prevent [injection attacks](https://www.w3schools.com/sql/sql_injection.asp) against your database. 

Since the SQL query to be executed is just a plain Python string, any data coming from an external source has the potential to manipulate that string into something it shouldn't be. Straight from the documentation:

<div class="alert alert-block alert-danger">
<b>Warning</b> Never, <b>never</b>, <b>NEVER</b> use Python string concatenation (+) or string parameters interpolation (%) to pass variables to a SQL query string. Not even at gunpoint.
</div>

There's something of a hacker urban legend of someone using an SQL injection attack to [trick a speed camera](https://hackaday.com/2014/04/04/sql-injection-fools-speed-traps-and-clears-your-record/) into deleting the entire table where it saves guilty license plate details. It actually inspired a true story of someone using a [custom license plate](https://news.sophos.com/en-us/2019/08/15/null-license-plate-gets-security-researcher-12k-in-tickets/) to try and hide in the system (it backfired!)

The `psycopg` library uses `%` symbols to represent parameters in SQL query strings, but this can sometimes conflict with existing syntax, so if you are using the `LIKE` operator, or trying to use `%` for modulo operation or some other purpose, make sure to read the [documentation](https://www.psycopg.org/docs/usage.html#the-problem-with-the-query-parameters) about how to apply it. Otherwise, it is a fairly simple application - the order of parameters matches the order of values in a tuple passed to `execute()` function as an additional parameter: 

In [12]:
cur.execute(
  '''
    INSERT INTO weather_data(date, city, temp, feels, description)
    VALUES (%s, %s, %s, %s, %s);
  ''', 
  (weather_date, weather_city, weather_temp, weather_feels, weather_description)
)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=db.bxzxrmyehjbtwvopzpyh.supabase.co database=postgres) at 0x1bd5e2bd910>

Since this is code that will also need to be hosted on AWS, it will need to eventually be written in regular `.py` files rather than `.ipynb`. It is best to take a functional approach and write functions for each step. 

## Your Task!

Now that you know how to send data to Railway, you will need to design a database to save the data you're requesting from the [Alpha Vantage stocks API](https://www.alphavantage.co/documentation/#currency-daily), and the data you're scraping from the [Financial Times](https://www.ft.com/). Then you can run your code that requests/scrapes, and populate your database on Railway! Remember you'll eventually be wanting to run those functions once per day to get the most recent results. Have a think about how you can adapt your code to be ready for that task as well.